# 日本酒のレコメンドシステム

====================================データの取得==============================================================

In [1]:
import urllib
from bs4 import BeautifulSoup
import re
import pyprind
#表記揺れの統一
import neologdn  

name_list=[]  #酒名
review_list=[]  #レビュー
all_list=[]  #余計な日本語以外全て
p=re.compile("[\\n\\r\\s0-9a-zA-Z]+")

pbar=pyprind.ProgBar(635)
#スクレイピングにより全てのページの日本酒を取得
for i in range(1,635):
    url_string="https://www.sakeno.com/voice/"
    if i!=1:
        url_string+="?p="+str(i)
    response=urllib.request.urlopen(url_string)
    soup=BeautifulSoup(response,'lxml')
    
    #銘柄の取得(表示用)
    for a in soup.find_all('a',href=re.compile("meigara/")):
        name_list.append(neologdn.normalize(a.string.replace("日本酒を探す","")))
    #レビューの取得(分析用)
    for a in soup.find_all('a',href=re.compile("voice/[0-9]+"),class_="newpostlink" ):            
        ary=[]
        ary=[]
        ary.append(p.sub("",(neologdn.normalize(a.string.replace('\u3000', '').replace("日本酒を探す","").replace('｡',"")))))
        all_list.append(ary)
    #レビューの取得(表示用)
    p=re.compile("[\\n\\r\\s]+")
    for a in soup.find_all('a',href=re.compile("voice/[0-9]+"),class_="newpostlink" ):
        ary=[]
        ary.append(p.sub("",a.string).replace('\u3000', '').replace("日本酒を探す","").replace('｡',""))
        review_list.append(ary)
    pbar.update()

0% [############################# ] 100% | ETA: 00:00:05

In [2]:
#解析データに銘柄名も追加
for i in range(len(all_list)):
    all_list[i][0]=str(all_list[i][0])+str(name_list[i])
    review_list[i][0]=str(review_list[i][0])+str(name_list[i])

レビューデータ

In [3]:
import pandas as pd
review_df=pd.DataFrame(review_list)
review_df.columns=['review']
print(review_df.isnull().sum(axis=0))
print(review_df.shape)

review    0
dtype: int64
(6340, 1)


銘柄データ

In [4]:
for i in range(len(name_list)):
    if i>len(name_list):
        break
    if name_list[i]=='':
        name_list.pop(i)

In [5]:
name_df=pd.DataFrame(name_list)
name_df.columns=['name']
print(name_df.isnull().sum(axis=0))
print(name_df.shape)

name    0
dtype: int64
(6340, 1)


結合

In [6]:
all_df=pd.concat([name_df, review_df], axis=1)
print(all_df.head(5))
mecab_all_df=all_df

  name                                             review
0   大七  東京駅で入手。ボトルに2013年醸造とあるので、6年熟成のようです。精米歩合69%、アルコー...
1   秋鹿  発売されてまもないので、キャップを開けたとたんに吹き出しました。味は食中酒です。濁り酒では甘...
2   十九  含んだアタックは強め、質感はトロリ、香味はライチ、余韻はキレ良く少しガス感含んで爽やかにキレ...
3  上喜元  純米にしては軽くてフルーティーさを感じるアルコールが15%のせいか味のせいか飲みやすいやや甘...
4   黒龍  黒龍のひやおろしを手に入れました。福井県産五百万石100%で精米歩合55%、アルコール度数1...


解析用レビューデータ

In [7]:
inf_df=pd.DataFrame(all_list)
inf_df.columns=['information']
print(inf_df.isnull().sum(axis=0))
print(inf_df.shape)

information    0
dtype: int64
(6340, 1)


=================================前処理==================================================================================

In [8]:
#絵文字の除去
import emoji
def remove_emoji(st):
    return ''.join(c for c in st if c not in emoji.UNICODE_EMOJI)
for i in range(len(all_list)):
    all_list[i]=remove_emoji(all_list[i])

-----------------------mecabによる形態素解析---------------------------------------------------------------

In [9]:
#Ver.Mecab
#名詞、動詞、形容詞、形容動詞の抽出
import numpy as np
from gensim.models import word2vec
import MeCab

#neologdの使用
tagger = MeCab.Tagger('-Ochasen C:\\Users\nakazawayuki\mecab-ipadic-neologd\bin')
#エラーの回避
tagger.parse('')

tmp=[]
tmp1=""
mecab_inf=[]
for text in all_list:
    node=tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] == u"名詞":
            tmp.append(node.surface)
            tmp1=" ".join(tmp)
            node = node.next
        elif node.feature.split(",")[0] == u"形容詞":
            tmp.append(node.feature.split(",")[6])
            tmp1=" ".join(tmp)
            node = node.next
        elif node.feature.split(",")[0] == u"動詞":
            tmp.append(node.feature.split(",")[6])
            tmp1=" ".join(tmp)
            node = node.next
        elif node.feature.split(",")[0] == u"形容動詞":
            tmp.append(node.feature.split(",")[6])
            tmp1=" ".join(tmp)
            node = node.next
        else:
            node = node.next       
    mecab_inf.append(tmp1)
    tmp=[]

In [10]:
#顔文字、記号の除去
word=re.compile("[０-９a-zA-Z！!%％「」、。．”’（）+,-?？@^0-9&℃♪\*\\\"\'±()__‼❗×“…※→○◎☆々〆・『』◆《》【】〒〇▼△↓÷← βαΘε‥〔￥]")
text =re.compile('<[^>]*>')
emotions = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')
for i in range(len(mecab_inf)):
    mecab_inf[i]=word.sub(" ",mecab_inf[i]) 
    mecab_inf[i]=text.sub(" ",mecab_inf[i])
    mecab_inf[i]=emotions.sub(" ",mecab_inf[i])
print(mecab_inf[0])

東京 駅 入手 ボトル 年 醸造 ある 年 熟成 よう 精米 歩合    アルコール 度 数 度 ヌル 燗 七 複雑 豊か 旨味 熟成 深み 増す 強い 広がる ラベル 燗 勧める 冷 ヌル 燗 試す 冷 熟成 独特 香り 感じる 老酒 よう 甘み アミノ酸 メイラード カラメル 香 それ 酸味 強い 感じる 温める 酸味 抑える られる 香り 抑える られる 熟成 する 日本 酒 さ 感じる 亀 酒 飲む やすい 七 旨味 七


In [11]:
#1文字だけの日本語を消去
mecab_information=[]
#words=re.compile("[ぁ-んァ-ン]")
for text in mecab_inf:
    mecab_information.append(text.split(" "))
for li in mecab_information:
    for i in range(len(li)):
        if len(li[i])==1:
            #li[i]=words.sub("",li[i])
            li[i]=li[i].replace(li[i],'')

In [12]:
#辞書の作成
from gensim import corpora,models
mecab_dictionary = corpora.Dictionary(mecab_information)
mecab_dictionary.filter_extremes()
#mecab_dictionary.filter_extremes(no_below=20)#単語数の制限
#保存
mecab_dictionary.save_as_text('mecab_dict.txt')  
#print(dictionary.token2id) #単語->idの辞書
print(len(mecab_dictionary))

2792


In [13]:
#コーパスの作成
#doc2bowで文書をbag-of-words形式に変換
mecab_corpus = [mecab_dictionary.doc2bow(text) for text in mecab_information]
print(len(mecab_corpus))

6340


In [14]:
#結合
tmp=[]
mecab_informations=[]
for text in mecab_information:
    for token in text:
        tmp.append(token)
        tmp1=" ".join(tmp)
    mecab_informations.append(tmp1)
    tmp=[]
mecab_wakati_df=pd.DataFrame(mecab_informations)
mecab_wakati_df.columns=['wakati']
mecab_all_df=pd.concat([mecab_all_df, mecab_wakati_df], axis=1)
print(mecab_all_df.head(5))

  name                                             review  \
0   大七  東京駅で入手。ボトルに2013年醸造とあるので、6年熟成のようです。精米歩合69%、アルコー...   
1   秋鹿  発売されてまもないので、キャップを開けたとたんに吹き出しました。味は食中酒です。濁り酒では甘...   
2   十九  含んだアタックは強め、質感はトロリ、香味はライチ、余韻はキレ良く少しガス感含んで爽やかにキレ...   
3  上喜元  純米にしては軽くてフルーティーさを感じるアルコールが15%のせいか味のせいか飲みやすいやや甘...   
4   黒龍  黒龍のひやおろしを手に入れました。福井県産五百万石100%で精米歩合55%、アルコール度数1...   

                                              wakati  
0  東京  入手 ボトル  醸造 ある  熟成 よう 精米 歩合    アルコール    ヌル ...  
1  発売 する れる ない キャップ 開ける とたん 吹き出す     濁り  甘い もの 多い...  
2  含む アタック 強める 質感 香味 ライチ 余韻 キレ 良い ガス  含む 爽やか キレ 良...  
3    する 軽い フルーティー  感じる アルコール   せい  せい 飲む やすい 甘い 後...  
4    ひる おろす  入れる 福井         精米 歩合    アルコール    軽い ...  


In [15]:
#簡易的な辞書の作成
mecab_dic={}
for ary in mecab_information:
    for word in ary:
        if word in mecab_dic:
            mecab_dic[word]+=1
        else:
            mecab_dic[word]=1
mecab_dic=sorted(mecab_dic.items(), key=lambda x:x[1],reverse=True)
#print(mecab_dic)

In [16]:
mecab_dic_df=pd.DataFrame(mecab_dic)
print(mecab_dic_df)

             0      1
0               96451
1           する   6072
2           飲む   3828
3           ある   3507
4           いる   2473
5           香り   2448
6          感じる   2163
7           よう   1736
8           思う   1679
9           良い   1668
10          吟醸   1510
11         味わい   1232
12          なる   1192
13          酸味   1147
14          甘い   1072
15          ない   1033
16          辛口   1017
17          日本   1014
18          感じ    979
19          れる    977
20          甘味    950
21          これ    907
22        美味しい    903
23          旨い    860
24          みる    808
25          こと    779
26          いい    778
27       アルコール    762
28          旨味    748
29          くる    695
...        ...    ...
10316       妻有      1
10317      手料理      1
10318      苗場山      1
10319       研く      1
10320       丹沢      1
10321       高校      1
10322       サメ      1
10323       心臓      1
10324      十日町      1
10325       白滝      1
10326       北口      1
10327     キタムラ      1
10328       真冬      1
10329     

=============================解析=================================================================

-------------------------gensim使用----------------------------------------------------------------------

In [17]:
tfidf = models.TfidfModel(mecab_corpus)
corpus_tfidf = tfidf[mecab_corpus]

In [18]:
#潜在ディリクレ配分
lda =models.LdaModel(corpus=corpus_tfidf, id2word=mecab_dictionary,
                             num_topics=35,random_state = 1,alpha = 0.01) #alphaの値が大きいと1つの文書が持つトピック数が多くなる

In [19]:
#トピックの抽出
for i in range(35):
    print("\n")
    print("="*80)
    print("TOPIC {0}\n".format(i))
    topic = lda.show_topic(i)
    for t in topic:
        print("{0:20s}{1}".format(t[0], t[1]))



TOPIC 0

乃誉                  0.04930901527404785
おすすめ                0.023339299485087395
埼玉                  0.018525011837482452
新酒                  0.01754440739750862
無し                  0.015765415504574776
文句                  0.012162013910710812
真っ白                 0.010696114972233772
にごり酒                0.01022858452051878
ここ                  0.009513255208730698
静岡                  0.009484927169978619


TOPIC 1

平成                  0.017648970708251
研究                  0.014365799725055695
詳しい                 0.013471606187522411
初孫                  0.013026507571339607
読む                  0.01254546269774437
きれい                 0.012383518740534782
はず                  0.011670637875795364
嫌味                  0.010768639855086803
かすか                 0.010375545360147953
入荷                  0.010019737295806408


TOPIC 2

料理                  0.0175942275673151
真澄                  0.013739422895014286
すき                  0.012447194196283817
飲む                  0.01244527567

In [20]:
def mecab_analysis(ary):
    #eologdの使用
    tagger = MeCab.Tagger('-Ochasen C:\\Users\nakazawayuki\mecab-ipadic-neologd\bin')
    #エラーの回避
    tagger.parse('')

    tmp=[]
    tmp1=""
    mecab_inf=[]
    for text in ary:
        node=tagger.parseToNode(text)
        while node:
            if node.feature.split(",")[0] == u"名詞":
                tmp.append(node.surface)
                tmp1=" ".join(tmp)
                node = node.next
            elif node.feature.split(",")[0] == u"形容詞":
                tmp.append(node.feature.split(",")[6])
                tmp1=" ".join(tmp)
                node = node.next
            elif node.feature.split(",")[0] == u"動詞":
                tmp.append(node.feature.split(",")[6])
                tmp1=" ".join(tmp)
                node = node.next
            else:
                node = node.next       
        mecab_inf.append(tmp1)
        tmp=[]

    word=re.compile("[０-９a-zA-Z！!%％「」、。．”’（）+,-?？@^0-9&℃♪\*\\\"\'±()__‼❗×“…※→○◎☆々〆・『』◆《》【】〒〇▼△↓÷← βαΘε‥〔￥]")
    text =re.compile('<[^>]*>')
    emotions = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')
    for i in range(len(mecab_inf)):
        mecab_inf[i]=word.sub(" ",mecab_inf[i]) 
        mecab_inf[i]=text.sub(" ",mecab_inf[i])
        mecab_inf[i]=emotions.sub(" ",mecab_inf[i])

    mecab_information=[]
    words=re.compile("[ぁ-んァ-ン]")
    for text in mecab_inf:
        mecab_information.append(text.split(" "))
    for li in mecab_information:
        for i in range(len(li)):
            if len(li[i])==1:
                   li[i]=li[i].replace(li[i],'')

    return mecab_information

-------------案1--------------------------------------

In [21]:
#和田龍登水のレビュー文
st="穏やかに舌の上に広がる甘味と旨味、甘すぎず旨すぎずバランスが良い。余韻のキレが良く、食中でグビッといきたい。色々合いそうですが、今日は茶碗蒸しと良く合った。"
seek=[]
seek.append(st)
#allow_updata：Dictionary内に存在しない語が文書内に含まれていた場合、その語を新たに追加し新単語を含めたBowを返す
seek_vec = mecab_dictionary.doc2bow(mecab_analysis(seek)[0])  #和田龍登水のベクトル
print(lda[seek_vec])
lda.print_topic(23)  #影響度が最も高い 

[(4, 0.3416026), (7, 0.24032749), (12, 0.3205167), (33, 0.08153048)]


'0.035*"うける" + 0.017*"割り" + 0.011*"目立つ" + 0.011*"本丸" + 0.010*"好く" + 0.010*"ちゃう" + 0.009*"味わい" + 0.009*"栽培" + 0.009*"価格" + 0.008*"飽く"'

In [21]:
#各レビューにトピックNo.の割り当て
topic_list=[]
for i in range(len(name_list)):
    seek_vec = mecab_dictionary.doc2bow(mecab_analysis(review_list[i])[0])
    tmp=lda[seek_vec]
    max=tmp[0][1]
    max_num=tmp[0][0]
    for j in range(len(tmp)):
        if max<tmp[j][1]:
            max_num=tmp[j][0]
    topic_list.append(max_num)

In [22]:
mecab_topic_number_df=pd.DataFrame(topic_list)
mecab_topic_number_df.columns=['topic number']
mecab_all_df=pd.concat([mecab_all_df, mecab_topic_number_df], axis=1)
#保存
mecab_all_df.to_csv('topic.csv', header=True, index=False)
print(mecab_all_df.head(5))

  name                                             review  \
0   大七  東京駅で入手。ボトルに2013年醸造とあるので、6年熟成のようです。精米歩合69%、アルコー...   
1   秋鹿  発売されてまもないので、キャップを開けたとたんに吹き出しました。味は食中酒です。濁り酒では甘...   
2   十九  含んだアタックは強め、質感はトロリ、香味はライチ、余韻はキレ良く少しガス感含んで爽やかにキレ...   
3  上喜元  純米にしては軽くてフルーティーさを感じるアルコールが15%のせいか味のせいか飲みやすいやや甘...   
4   黒龍  黒龍のひやおろしを手に入れました。福井県産五百万石100%で精米歩合55%、アルコール度数1...   

                                              wakati  topic number  
0  東京  入手 ボトル  醸造 ある  熟成 よう 精米 歩合    アルコール    ヌル ...            25  
1  発売 する れる ない キャップ 開ける とたん 吹き出す     濁り  甘い もの 多い...             5  
2  含む アタック 強める 質感 香味 ライチ 余韻 キレ 良い ガス  含む 爽やか キレ 良...            25  
3    する 軽い フルーティー  感じる アルコール   せい  せい 飲む やすい 甘い 後...            17  
4    ひる おろす  入れる 福井         精米 歩合    アルコール    軽い ...            12  


次回ここから開始可能

In [23]:
import pandas as pd
mecab_topic_number_df=pd.read_csv('topic.csv',header=None,encoding='CP932')
mecab_topic_number_df.drop(mecab_topic_number_df.index[[1]])
mecab_topic_number_df.columns=['name','review','wakati','topic number']
mecab_topic_number_df=mecab_topic_number_df.drop(mecab_topic_number_df.index[[0]])
mecab_topic_number_df['topic number']=mecab_topic_number_df['topic number'].astype(int)
print(mecab_topic_number_df.head(5))

  name                                             review  \
1   大七  東京駅で入手。ボトルに2013年醸造とあるので、6年熟成のようです。精米歩合69%、アルコー...   
2   秋鹿  発売されてまもないので、キャップを開けたとたんに吹き出しました。味は食中酒です。濁り酒では甘...   
3   十九  含んだアタックは強め、質感はトロリ、香味はライチ、余韻はキレ良く少しガス感含んで爽やかにキレ...   
4  上喜元  純米にしては軽くてフルーティーさを感じるアルコールが15%のせいか味のせいか飲みやすいやや甘...   
5   黒龍  黒龍のひやおろしを手に入れました。福井県産五百万石100%で精米歩合55%、アルコール度数1...   

                                              wakati  topic number  
1  東京  入手 ボトル  醸造 ある  熟成 よう 精米 歩合    アルコール    ヌル ...            25  
2  発売 する れる ない キャップ 開ける とたん 吹き出す     濁り  甘い もの 多い...             5  
3  含む アタック 強める 質感 香味 ライチ 余韻 キレ 良い ガス  含む 爽やか キレ 良...            25  
4    する 軽い フルーティー  感じる アルコール   せい  せい 飲む やすい 甘い 後...            17  
5    ひる おろす  入れる 福井         精米 歩合    アルコール    軽い ...            12  


In [24]:
def mecab_analysis(ary):
    #eologdの使用
    tagger = MeCab.Tagger('-Ochasen C:\\Users\nakazawayuki\mecab-ipadic-neologd\bin')
    #エラーの回避
    tagger.parse('')

    tmp=[]
    tmp1=""
    mecab_inf=[]
    for text in ary:
        node=tagger.parseToNode(text)
        while node:
            if node.feature.split(",")[0] == u"名詞":
                tmp.append(node.surface)
                tmp1=" ".join(tmp)
                node = node.next
            elif node.feature.split(",")[0] == u"形容詞":
                tmp.append(node.feature.split(",")[6])
                tmp1=" ".join(tmp)
                node = node.next
            elif node.feature.split(",")[0] == u"動詞":
                tmp.append(node.feature.split(",")[6])
                tmp1=" ".join(tmp)
                node = node.next
            else:
                node = node.next       
        mecab_inf.append(tmp1)
        tmp=[]

    word=re.compile("[０-９a-zA-Z！!%％「」、。．”’（）+,-?？@^0-9&℃♪\*\\\"\'±()__‼❗×“…※→○◎☆々〆・『』◆《》【】〒〇▼△↓÷← βαΘε‥〔￥]")
    text =re.compile('<[^>]*>')
    emotions = re.compile('(?::|;|=)(?:-)?(?:\)|\(|D|P)')
    for i in range(len(mecab_inf)):
        mecab_inf[i]=word.sub(" ",mecab_inf[i]) 
        mecab_inf[i]=text.sub(" ",mecab_inf[i])
        mecab_inf[i]=emotions.sub(" ",mecab_inf[i])

    mecab_information=[]
    for text in mecab_inf:
        mecab_information.append(text.split(" "))
    for li in mecab_information:
        for i in range(len(li)):
            if len(li[i])==1:
                   li[i]=li[i].replace(li[i],'')

    return mecab_information

=========================リコメンド機能==============================================================================

In [26]:
import numpy as np
from gensim.models import word2vec
import MeCab
import re

st="爽やかな香り"
seek=[]
seek.append(st)
seek_vec = mecab_dictionary.doc2bow(mecab_analysis(seek)[0]) 
#入力情報をクラスタリング -> トピック番号の取得
tmp=lda[seek_vec]
max=tmp[0][1]
topic_num=tmp[0][0]
for j in range(len(tmp)):
     if max<tmp[j][1]:
        topic_num=tmp[j][0]
topic_df=mecab_topic_number_df[mecab_topic_number_df['topic number'].isin([topic_num])]
print(topic_df.head(5))

     name                                             review  \
2      十九  含んだアタックは強め、質感はトロリ、香味はライチ、余韻はキレ良く少しガス感含んで爽やかにキレ...   
21   銀嶺立山                      私（妻)口当たりよすぎてたくさんのんでしまう。旦那うまい。   
79     王祿  含むとスッと入り、ほんのり米の甘味と締まるスダチの酸。味わいを感じたと思ったら、もういなくな...   
138    仙介  仙介は以前飲んだ純米大吟醸には、フルーティな香りに驚かされましたが、この白麹無濾過生酒原酒に...   
142    国権  とてもバランスが良いですね。適度な香りと上品なうまみがあってアテ無しでもおいしく飲めます。値...   

                                                wakati  topic number  
2    含む アタック 強める 質感 香味 ライチ 余韻 キレ 良い ガス  含む 爽やか キレ 良...            34  
21                         口当たり よす ぎてたくさんのんでしまう 旦那 うまい            34  
79   含む スッ 入る  甘味 締まる スダチ  味わい 感じる 思う いる なる いる   抜群...            34  
138  仙介 以前 飲む  吟醸 フルーティ 香り 驚かす れる   濾過 生酒 原酒 ビックリ 兵...            34  
142  バランス 良い 適度 香り 上品 うまみ ある アテ 無し おいしい 飲める 値段 手頃 こ...            34  


In [27]:
#類似度の計算
from gensim import corpora, models, similaritie
wakati_ary=topic_df['wakati'].tolist()
simil_ary=[]
for token in wakati_ary:
    simil_ary.append(token.split())

#コーパスの作成
#doc2bowで文書をbag-of-words形式に変換
simil_corpus = [mecab_dictionary.doc2bow(text) for text in simil_ary]
#if len(simil_corpus) !=0:
index = similarities.MatrixSimilarity(lda[simil_corpus])
vec_lda = lda[seek_vec]
sims=[]
sims = index[vec_lda]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
display_name=topic_df['name'].values.tolist()
display_review=topic_df['review'].values.tolist()
if len(display_name)<3:
    for i in range(len(display_name)):
        print('あなたにおすすめの日本酒:  '+display_name[int(sims[i][0])]+'\n'+'レビュー:  '+display_review[int(sims[i][0])]+'\n')
else:
    for i in range(3):
        print('あなたにおすすめの日本酒:  '+display_name[int(sims[i][0])]+'\n'+'レビュー:  '+display_review[int(sims[i][0])]+'\n')
#print(sims)

あなたにおすすめの日本酒:  銀嶺立山
レビュー:  私（妻)口当たりよすぎてたくさんのんでしまう。旦那うまい。

あなたにおすすめの日本酒:  如空
レビュー:  甘味のある、香りと旨味の多い日本酒。なつかしい味。おいしい。

あなたにおすすめの日本酒:  菊盛
レビュー:  値段はとても安いです。でも、肝心の味わいは･･･。残念!!



=======================シリアライズ============================================================================

In [55]:
import pickle
import os
dest=os.path.join('sakeclassifier','pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
#辞書のシリアライズ
pickle.dump(mecab_dictionary,
           open(os.path.join(dest,'dictionary.pkl'),'wb'),
           protocol=4)
#分類器のシリアライズ
pickle.dump(lda,
           open(os.path.join(dest,'lda.pkl'),'wb'),
           protocol=4)

===================デシリアライズ====================================================================================

In [57]:
import pickle
import os
lda=pickle.load(open(os.path.join('pkl_objects','lda.pkl'),'rb'))
mecab_dictionary=pickle.load(open(os.path.join('pkl_objects','dictionary.pkl'),'rb'))

==================完了==============================================================================================

--------------類似度のみVer(情報ネットワーク論用)-------------------------------------------------

In [25]:
from gensim import corpora, models, similarities
import numpy as np
st="高級純米酒"
seek=[]
seek.append(st)
seek_vec = mecab_dictionary.doc2bow(mecab_analysis(seek)[0]) 

index=similarities.MatrixSimilarity(mecab_corpus,num_features=len(mecab_dictionary))
simil=index[seek_vec]
print(str(np.argmax(similarities))+','+str(simil[np.argmax(simil)]))
#print(len(simil))
for i in range(len(simil)):
    if simil[i]>0:
        print(simil[i])

tuple_list=[]
for i in range(len(simil)):
    tuple_list.append((simil[i],i))
tuple_list=sorted(tuple_list,reverse=True)
print(tuple_list)
#print(np.argmax(simil))

0,0.37796447
0.16222142
0.11111111
0.18898223
0.11867817
0.23570226
0.13245323
0.13363062
0.1796053
0.2236068
0.14002801
0.19611613
0.12126781
0.18257418
0.19245009
0.2
0.15249857
0.120385855
0.13736056
0.23570226
0.2085144
0.17149858
0.16666667
0.13363062
0.21320072
0.37796447
0.11785113
0.14002801
[(0.37796447, 6027), (0.23570226, 4427), (0.23570226, 1975), (0.2236068, 3282), (0.21320072, 5476), (0.2085144, 4582), (0.2, 4340), (0.19611613, 3630), (0.19245009, 4308), (0.18898223, 930), (0.18257418, 4171), (0.1796053, 2809), (0.17149858, 4899), (0.16666667, 4955), (0.16222142, 44), (0.15249857, 4403), (0.14002801, 6288), (0.14002801, 3461), (0.13736056, 4417), (0.13363062, 5011), (0.13363062, 2619), (0.13245323, 2401), (0.12126781, 3738), (0.120385855, 4408), (0.11867817, 1107), (0.11785113, 6057), (0.11111111, 875), (0.0, 6339), (0.0, 6338), (0.0, 6337), (0.0, 6336), (0.0, 6335), (0.0, 6334), (0.0, 6333), (0.0, 6332), (0.0, 6331), (0.0, 6330), (0.0, 6329), (0.0, 6328), (0.0, 6327), (0

In [26]:
print(mecab_topic_number_df['name'][np.argmax(simil)+1])
print(mecab_topic_number_df['review'][np.argmax(simil)+1])
print(mecab_topic_number_df['name'][tuple_list[0][1]+1])
print(mecab_topic_number_df['review'][tuple_list[0][1]+1])

太平山
新鮮さの中にも濃醇な旨味、香り穏やかに高級感がある。
太平山
新鮮さの中にも濃醇な旨味、香り穏やかに高級感がある。


======================シリアライズ===================================================

In [27]:
import pickle
import os
dest=os.path.join('','pkl_objects1')
if not os.path.exists(dest):
    os.makedirs(dest)
#辞書のシリアライズ
pickle.dump(mecab_dictionary,
           open(os.path.join(dest,'dictionary.pkl'),'wb'),
           protocol=4)
#コーパスのシリアライズ
pickle.dump(mecab_corpus,
           open(os.path.join(dest,'corpus.pkl'),'wb'),
           protocol=4)

=======================デシリアライズ========================================================

In [29]:
import pickle
import os
mecab_corpus=pickle.load(open(os.path.join('pkl_objects1','corpus.pkl'),'rb'))
mecab_dictionary=pickle.load(open(os.path.join('pkl_objects1','dictionary.pkl'),'rb'))

------------終了-------------------------------------------------

==================精度評価======================================================================================

In [44]:
import numpy as np
from gensim.models.ldamodel import CoherenceModel

for i in range(20, 100,5):
    lda = models.LdaModel(corpus = corpus_tfidf, id2word = mecab_dictionary, num_topics = i, alpha = 0.01, random_state = 1)

    cm = CoherenceModel(model = lda, corpus = mecab_corpus, coherence = 'u_mass')
    coherence = cm.get_coherence()

    perwordbound = lda.log_perplexity(mecab_corpus)
    perplexity = np.exp2(-perwordbound)

    print(f"num_topics = {i}, coherence = {coherence}, perplexity = {perplexity}")

num_topics = 20, coherence = -9.057373369686738, perplexity = 253.30237629126287
num_topics = 25, coherence = -10.222401395286234, perplexity = 306.1966783691849
num_topics = 30, coherence = -10.011521931513311, perplexity = 342.21279409133143
num_topics = 35, coherence = -11.297242670286657, perplexity = 406.545733489273
num_topics = 40, coherence = -11.482465307027558, perplexity = 466.7156080356021
num_topics = 45, coherence = -12.14505757160668, perplexity = 568.4310747300158
num_topics = 50, coherence = -11.894876247578484, perplexity = 725.6089912843377
num_topics = 55, coherence = -11.999322400084434, perplexity = 1006.730767998302
num_topics = 60, coherence = -11.525368637886432, perplexity = 1606.959932597497
num_topics = 65, coherence = -9.458863209318627, perplexity = 3343.452102124621
num_topics = 70, coherence = -6.578151896915825, perplexity = 12858.563412825775
num_topics = 75, coherence = -7.309769338190023, perplexity = 168071.60158663275
num_topics = 80, coherence = -

perplexity...予測精度(値が小さい方が良い)   coherence...一貫性(値が大きい方が良い)

In [64]:
from statistics import mean

for t in range(20, 100, 5):
    for n in range(1, 10, 2):
        a = n / 100

        lda = models.LdaModel(corpus = corpus_tfidf, id2word = mecab_dictionary, num_topics = t, alpha = a, random_state = 1)

        # 文書の平均トピック数を算出
        r = mean([len(lda[c]) for c in corpus_tfidf])

        print(f"num_topics = {t}, alpha = {a}, mean = {r}")

num_topics = 20, alpha = 0.01, mean = 2.682148499210111
num_topics = 20, alpha = 0.03, mean = 4.544075829383886
num_topics = 20, alpha = 0.05, mean = 13.36129541864139
num_topics = 20, alpha = 0.07, mean = 19.748973143759873
num_topics = 20, alpha = 0.09, mean = 20
num_topics = 25, alpha = 0.01, mean = 2.9695102685624013
num_topics = 25, alpha = 0.03, mean = 4.73996840442338
num_topics = 25, alpha = 0.05, mean = 14.397314375987362
num_topics = 25, alpha = 0.07, mean = 23.98309636650869
num_topics = 25, alpha = 0.09, mean = 25
num_topics = 30, alpha = 0.01, mean = 3.1913112164296997
num_topics = 30, alpha = 0.03, mean = 4.717851500789889
num_topics = 30, alpha = 0.05, mean = 14.458609794628751
num_topics = 30, alpha = 0.07, mean = 27.131279620853082
num_topics = 30, alpha = 0.09, mean = 29.992575039494472
num_topics = 35, alpha = 0.01, mean = 3.3925750394944707
num_topics = 35, alpha = 0.03, mean = 4.650236966824645
num_topics = 35, alpha = 0.05, mean = 13.8173775671406
num_topics = 35,

In [161]:
print(len(simil_corpus))

94


In [162]:
print(simil_corpus)

[[(6, 2), (9, 2), (11, 1), (44, 1), (301, 1), (346, 1), (1019, 1), (1020, 1)], [(0, 3), (27, 1), (41, 1), (44, 1), (52, 1), (69, 2), (78, 1), (79, 1), (88, 1), (98, 1), (152, 1), (161, 1), (178, 1), (185, 1), (255, 1), (260, 1), (266, 1), (307, 1), (326, 1), (433, 1), (473, 1), (602, 1), (607, 1), (694, 1), (786, 1), (827, 1), (1106, 2)], [(10, 1), (23, 1), (26, 1), (41, 1), (55, 1), (98, 1), (99, 1), (312, 1), (570, 1), (740, 1), (902, 1), (936, 1), (1072, 1), (1106, 2)], [(0, 2), (9, 1), (10, 1), (20, 1), (44, 2), (52, 1), (55, 1), (60, 1), (61, 1), (165, 1), (188, 1), (194, 1), (251, 1), (575, 1), (677, 1), (833, 1), (969, 1)], [(0, 2), (4, 1), (9, 3), (44, 1), (58, 1), (60, 2), (68, 1), (71, 2), (183, 1), (211, 1), (243, 1), (250, 1), (251, 1), (252, 1), (340, 1), (385, 1), (453, 1), (471, 1), (593, 1), (598, 2), (627, 1), (669, 1), (1451, 1), (1728, 1)], [(4, 1), (6, 1), (13, 1), (18, 1), (23, 2), (44, 1), (55, 2), (59, 1), (112, 1), (146, 1), (173, 1), (238, 1), (253, 2), (471, 1

In [163]:
print(len(mecab_corpus))

6330


In [32]:
print(lda[simil_corpus])